## GBDT+LR代码分析

In [1]:
# Scikit-learn实现

In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier

In [4]:
gbdt = GradientBoostingClassifier(n_estimators=50,random_state=10,subsample = 0.6,max_depth=7,min_samples_split=900)

In [8]:
gbm1.fit.(X_train,Y_train)
train_new_feature = gbm1.apply(X_train)
train_new_feature = train_new_feature.reshape(-1,50) # 子树为50个
enc = OneHotEncoder()
enc.fit(train_new_feature)
# # 每一个属性的最大取值数目
# print('每一个特征的最大取值数目:', enc.n_values_)
# print('所有特征的取值数目总和:', enc.n_values_.sum())

SyntaxError: invalid syntax (<ipython-input-8-55bb600a4a2a>, line 1)

In [ ]:
train_new_feature2 = np.array(enc.transform(train_new_feature).toarray())

划重点：
5.1.1 model.apply(X_train)的用法
model.apply(X_train)返回训练数据X_train在训练好的模型里每棵树中所处的叶子节点的位置（索引）

5.1.2 sklearn.preprocessing 中OneHotEncoder的使用
除了pandas中的 get_dummies()，sklearn也提供了一种对Dataframe做One-hot的方法。

OneHotEncoder() 首先fit() 过待转换的数据后，再次transform() 待转换的数据，就可实现对这些数据的所有特征进行One-hot 操作。

由于transform() 后的数据格式不能直接使用，所以最后需要使用.toarray() 将其转换为我们能够使用的数组结构

In [5]:
# lightgbm的实现

In [7]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 64,
    'num_trees': 100,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}


# number of leaves,will be used in feature transformation
num_leaf = 64

print('Start training...')
# train
gbm = lgb.train(params=params,
                train_set=lgb_train,
                valid_sets=lgb_train, )


print('Start predicting...')
# y_pred分别落在100棵树上的哪个节点上
y_pred = gbm.predict(x_train, pred_leaf=True)
y_pred_prob = gbm.predict(x_train)

result = []
threshold = 0.5
for pred in y_pred_prob:
    result.append(1 if pred > threshold else 0)
print('result:', result)

for i in range(0, len(y_pred)):
    # temp表示在每棵树上预测的值所在节点的序号（0,64,128,...,6436 为100棵树的序号，中间的值为对应树的节点序号）
    temp = np.arange(len(y_pred[0])) * num_leaf + np.array(y_pred[i])
    # 构造one-hot 训练数据集
    transformed_training_matrix[i][temp] += 1

    
y_pred = gbm.predict(x_test, pred_leaf=True)
print('Writing transformed testing data')
transformed_testing_matrix = np.zeros([len(y_pred), len(y_pred[1]) * num_leaf], dtype=np.int64)
for i in range(0, len(y_pred)):
    temp = np.arange(len(y_pred[0])) * num_leaf + np.array(y_pred[i])
    # 构造one-hot 测试数据集
    transformed_testing_matrix[i][temp] += 1

Start training...


NameError: name 'lgb' is not defined

划重点：
5.2.1 params 字典里超参数的设置
因为是二分类问题，所以设置 {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss'}}，然后设置树的个数及每棵树的叶子结点个数{'num_leaves': 64,'num_trees': 100}

5.2.2 model.predict(x_train, pred_leaf=True)
使用